In [43]:
import spacy
nlp = spacy.load('en')

In [ ]:
# key lemma: premiere, screening, promotion
#patter1 for lemma noun:
    # attachment with "of" for key lemma is movie name
    # find attached verb with lemma
    # get location of that verb
    # get temoral of that verb
#pattern2 for lemma verb:
    # get location of lemma verb
    # get temporal of lemma verb
    # subject of lemma verb is movie (token -> noun phrase)

In [49]:
s1 = "The premiere of IronMan was held in the Greater Union Theater at George Street, Sydney, on April 14, 2008."
s2 = "Black Panther premiered in Riyadh, Saudi Arabia on April 18, 2018"
s3 = "Ironman screened at the Toronto International Film Festival, the San Sebastián International Film Festival, and the Zurich Film Festival in September 2018."
s4 = "A screening of the Captain America was held during the 2010 Fordham University Law School Film festival, Supreme Court Justice Sonia Sotomayor stated that seeing 12 Angry Men while she was in college influenced her decision to pursue a career in law."
s5 = "The Ironman premiered at the Sundance Film Festival on January 21, 2018, and was theatrically released in the United States on August 24, 2018, by Screen Gems."
s6 = "Harry Porter had its world premiere at the Venice Film Festival on August 31, 2018."
#problem to solve: conj statement parsing issue
#accuracy: 65%

In [51]:
sent = s1
parse(sent)

---- Promotional Event ----
Movie Name:IronMan
Event type:premiere
Location:the Greater Union Theater
Time:April 14, 2008


In [2]:
def parse(sent):
    promotionNoun = findPromotionNoun(sent)
    promotionVerb = findPromotionVerb(sent)
    
    result = None
    if promotionNoun != None:
        result = parseNounPattern(sent, promotionNoun)
    elif promotionVerb != None :
        result = parseVerbPattern(sent, promotionVerb)
        
    printResult(result)

In [21]:
def findPromotionNoun(sent):
    for token in nlp(sent):
        if token.lemma_ in ["premiere","screening", "promotion"]:
            if token.pos_ == "NOUN":
                return token.lemma_
    return None

In [22]:
def findPromotionVerb(sent):
    for token in nlp(sent):
        if token.lemma_ in ["premier","screen", "promote"]:
            if token.pos_ == "VERB":
                return token.lemma_
    return None

In [23]:
def parseNounPattern(sent, promotionNoun):
    for token in nlp(sent):
        #noun
        if token.lemma_ == promotionNoun:
            if token.head.pos_ == "VERB":
                location = getLocationOfVerbToken(sent, token.head)
                if location == "<no-match>":
                    location = getLocationOfVerbToken(sent, token)
                time = getTimeOfVerbToken(sent, token.head)
                movie = getEntityRelatedByOF(sent, token)
                if movie == "<no-match>":
                    movie = getSubjectEntityOfVerbToken(sent, token.head)
                return [movie,promotionNoun,location, time]
    return None

In [24]:
def parseVerbPattern(sent, promotionVerb):
    for token in nlp(sent):
        #verb
        if token.lemma_ == promotionVerb:
            location = getLocationOfVerbToken(sent, token)
            time = getTimeOfVerbToken(sent, token)
            movie = getSubjectEntityOfVerbToken(sent, token)
            return [movie,promotionVerb,location, time]
    return None

In [25]:
def getSubjectEntityOfVerbToken(sent, token):
    for child in token.children:
        if "subj" in str(child.dep_):
            entity = getEntityThatContainsToken(sent, child)
            if entity == None:
                entity = getPhraseThatContainsToken(sent, child)
            return entity

In [27]:
def getEntityRelatedByOF(sent, token):
    for child in token.children: 
                #usually movie name for noun attached with "on"
                if child.text in ["of"]:
                    #['of', 'prep', 'premiere', 'NOUN', [IronMan]]
                    for subChild in child.children:
                        entity = getEntityThatContainsToken(sent, subChild)
                        if(entity != None):
                            return entity
    return "<no-match>"

In [28]:
def getTimeOfVerbToken(sent, verbToken):
    for child in verbToken.children: 
                #usually release time starts with "on"
                if child.text in ["on","at"]:
                    #usually "on"'s child is [August] 
                    for subChild in child.children:
                        temporalEntity = getTemporalThatContainsToken(sent, subChild)
                        if(temporalEntity != None):
                            return temporalEntity
    return "<no-match>"

In [29]:
def getLocationOfVerbToken(sent, verbToken):
    for child in verbToken.children: 
        #usually release location starts with "in"
        if child.text in ["in","throughout", "at"]:
            for subChild in child.children:
                GPEEntity = getLocationThatContainsToken(sent, subChild)
                if(GPEEntity != None):
                    return GPEEntity
    return "<no-match>"

In [30]:
def getLocationThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and ent.label_ != "TIME" and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return None

In [31]:
import re
def calculateStartPositionOfToken(statement, targetToken):
    """
    this should return start position of the token
    """
    matches = [m.start() for m in re.finditer(targetToken.text, statement)]
    if(len(matches)==1):
        return matches[0]
    

    huristicPosition = 0
    doc = nlp(statement)
    
    i = 0
    for token in doc:
        if i < targetToken.i:
            huristicPosition = huristicPosition + len(token.text)+1
            i = i+1
        else:
            break
        
    distances = []
    for match in matches:
        distances.append(abs(match-huristicPosition))
        
    minIndex = distances.index(min(distances))
    
    return matches[minIndex]

In [32]:
def getTemporalThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and ent.label_ == "DATE" and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return None

In [33]:
def getEntityThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return None

In [34]:
def getPhraseThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for chunk in doc.noun_chunks:
        if(token.text in chunk.text and tokenStartPos >= chunk.start_char and tokenStartPos <= chunk.end_char):
            return chunk.text
    return None

In [37]:
def printResult(result):
    print("---- Promotional Event ----")
    if(result!=None):
        print("Movie Name:"+str(result[0]))
        print("Event type:"+str(result[1]))
        print("Location:"+str(result[2]))
        print("Time:"+str(result[3]))
    else:
        print("Could not parse")